# Generating a maritime data_set using a global data_set about the oceans (Shom France)
### It needs a file of 900mb, it's not on github

# Parameters

In [1]:
# Number of earth points = num_rows*num_col
lower_bound_lat, upper_bound_lat = 44.400000, 45.000000
lower_bound_long, upper_bound_long = -1.400000, -0.800000
linear_density = 150 # Number of point in every row

var_col, var_row = upper_bound_long - lower_bound_long, upper_bound_lat - lower_bound_lat

# Generating earth data_set using an API (OpenTopoData)

### Functions

In [2]:
import time
import requests


def obtain_lat_and_long(limites_lat=[lower_bound_lat, upper_bound_lat],limites_long=[upper_bound_long, lower_bound_long],num_rows=int(linear_density/var_row),num_col=int(linear_density/var_col)):
    all_coords=[]
    for i in range(num_rows):
        lat=float('%.5f'%(limites_lat[1]-(i)*(limites_lat[1]-limites_lat[0])/num_rows))
        for j in range (num_col):
            long=float('%.5f'%(limites_long[1]-(j)*(limites_long[1]-limites_long[0])/num_col))
            all_coords.append([lat, long])
            
    return all_coords


def get_elevation(coords : list()):
    time.sleep(1.2)
    query =""
    for coord in coords:
        query += '{0},{1}'.format(coord[0],coord[1])
        query += '|'
    query = query[:-1]
    r = requests.get('https://api.opentopodata.org/v1/eudem25m?locations='+query, headers={"Content-Type": "application/json", "Accept": "application/json"}, timeout=1000)
    result = r.json()
    elevations = [res['elevation'] for res in result['results']]
    return elevations


# Fonction to obtain the topography data of all the points (lat,long)
def dataset_generator(coords,file_name):
    depth=[]
    file_data = open(file_name,'a')
    iter_max=int(len(coords)/100)
    print("Number of iterations:", iter_max)
    for i in range(iter_max):
        print("Iter",i+1,"Progress",100*(i+1)/iter_max,"%")
        depth=get_elevation(coords[i*100:(i+1)*100])
        for j in range(len(depth)):
            if depth[j]!=None:
                file_data.write(str(coords[i*100+j][1])+','+str(coords[i*100+j][0])+','+str(depth[j])+'\n')

### Cropping maritime_data

In [3]:
import pandas as pd


maritime_data = pd.read_csv("./atlantic_data.txt",sep='\t')
maritime_data.columns = ["long", "lat", "depth"]

maritime_data["long"] = maritime_data["long"].astype(float)
maritime_data["lat"] = maritime_data["lat"].astype(float)
maritime_data["depth"] = maritime_data["depth"].astype(float)

maritime_data.drop(maritime_data[maritime_data.lat<lower_bound_lat].index, inplace=True)
maritime_data.drop(maritime_data[maritime_data.lat>upper_bound_lat].index, inplace=True)
maritime_data.drop(maritime_data[maritime_data.long>upper_bound_long].index, inplace=True)
maritime_data.drop(maritime_data[maritime_data.long<lower_bound_long].index, inplace=True)
maritime_data.to_csv(r'./maritime_data.csv', index=False, header=True)

### Generate the topography data set

In [4]:
file_data = open("./earth_data.csv",'w')
file_data.write("long,lat,depth\n")
file_data.close()

coords=obtain_lat_and_long()
dataset_generator(coords, "./earth_data.csv")

Number of iterations: 622
Iter 1 Progress 0.1607717041800643 %
Iter 2 Progress 0.3215434083601286 %
Iter 3 Progress 0.48231511254019294 %
Iter 4 Progress 0.6430868167202572 %
Iter 5 Progress 0.8038585209003215 %
Iter 6 Progress 0.9646302250803859 %
Iter 7 Progress 1.1254019292604502 %
Iter 8 Progress 1.2861736334405145 %
Iter 9 Progress 1.4469453376205788 %
Iter 10 Progress 1.607717041800643 %
Iter 11 Progress 1.7684887459807075 %
Iter 12 Progress 1.9292604501607717 %
Iter 13 Progress 2.090032154340836 %
Iter 14 Progress 2.2508038585209005 %
Iter 15 Progress 2.4115755627009645 %
Iter 16 Progress 2.572347266881029 %
Iter 17 Progress 2.733118971061093 %
Iter 18 Progress 2.8938906752411575 %
Iter 19 Progress 3.054662379421222 %
Iter 20 Progress 3.215434083601286 %
Iter 21 Progress 3.3762057877813505 %
Iter 22 Progress 3.536977491961415 %
Iter 23 Progress 3.697749196141479 %
Iter 24 Progress 3.8585209003215435 %
Iter 25 Progress 4.019292604501608 %
Iter 26 Progress 4.180064308681672 %
Iter

### Size information

In [5]:
earth_data = pd.read_csv("./earth_data.csv", sep=",", header=3)
earth_data.columns = ["long", "lat", "depth"]
print("Number of points earth_data:",len(earth_data['long']))

maritime_data = pd.read_csv("./maritime_data.csv",sep=",")
maritime_data.columns = ["long", "lat", "depth"]
print("Number of points maritime_data:",len(maritime_data["long"]))

Number of points earth_data: 42333
Number of points maritime_data: 116294
